In [74]:
#1
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

In [75]:
park = Customer('park',100)
park

Customer(name='park', fidelity=100)

In [76]:
class LineItem:
    def __init__(self,product,quantity,price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

In [77]:
class Order:
    def __init__(self,customer,cart,promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    def total(self):
        if not hasattr(self,'__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())



In [78]:
class Promotion(ABC):
    @abstractmethod
    def discount(self,order):
        pass

In [79]:
class FidelityPromo(Promotion):
    def discount(self,order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0


class BulkItemPromo(Promotion):
    def discount(self,order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
        return discount

class LargeOrderPromo(Promotion):
    def discount(self,order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0

In [80]:
#2
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [
LineItem('banana', 4, .5),
LineItem('apple', 10, 1.5),
LineItem('watermellon', 5, 5.0)]


In [81]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [82]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [83]:
banana_cart = [LineItem('banana', 30, .5), LineItem('apple', 10, 1.5)]


In [84]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [85]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

In [86]:
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [87]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

In [88]:
#3
class Order:
  def __init__(self, customer, cart, promotion=None):
    self.customer = customer
    self.cart = list(cart)
    self.promotion = promotion

  def total(self):
    if not hasattr(self, '__total'):
      self.__total = sum(item.total() for item in self.cart)
    return self.__total

  def due(self):
    if self.promotion is None:
      discount = 0
    else:
      discount = self.promotion(self)
    return self.total() - discount

  def __repr__(self):
    fmt = '<Order total: {:.2f} due: {:.2f}>'
    return fmt.format(self.total(), self.due())

In [89]:
def fidelity_promo(order):
  return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * 0.1
  return discount

def large_order_promo(order):
  distinct_items = {item.product for item in order.cart}
  if len(distinct_items) >= 10:
    return order.total() * 0.07
  return 0

In [90]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [91]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [92]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [93]:
Order(ann, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [94]:
#4
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
  return max(promo(order) for promo in promos)

In [95]:
#5
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [96]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [97]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [98]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'ABC', 'abstractmethod', 'namedtuple', 'customer', '_i2', '_i3', 'Customer', '_i4', '_i5', '_i6', 'park', '_6', '_i7', 'LineItem', '_i8', '_i9', '_9', '_i10', '_i11', 'Order', '_i12', 'Promotion', '_i13', 'FidelityPromo', 'BulkItemPromo', 'LargeOrderPromo', '_i14', 'joe', 'ann', 'cart', '_i15', '_15', '_i16', '_16', '_i17', 'banana_cart', '_i18', '_18', '_i19', 'long_order', '_i20', '_20', '_i21', '_21', '_i22', '_i23', 'fidelity_promo', 'bulk_item_promo', 'large_order_promo', '_i24', '_24', '_i25', '_25', '_i26', '_26', '_i27', '_27', '_i28', 'promos', 'best_promo', '_i29', '_29', '_i30', '_30', '_i31', '_31', '_i32', '_32', '_i33', '_33', '_i34', '_34', '_i35', 'f1', '_i36', 'b', '_i37', '_i38', '_38', '_i39', '_i40', '_i41', '_i42', '_i43', '_i44', '_44', '_i45', '_45',

In [99]:
globals()['best_promo']

<function __main__.best_promo(order)>

In [100]:
#6
promos = [globals()[name] for name in globals()
if name.endswith('_promo')
and name != 'best_promo']
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]

In [101]:
#07-1
def deco(func):
  def inner():
    print('running inner()')
  return inner

In [102]:
@deco
def target():
  print('running target()')
target()

running inner()


In [103]:
target

<function __main__.deco.<locals>.inner()>

In [104]:
#07-2
registry = []
def register(func):
  print('running register(%s)' % func)
  registry.append(func)
  return func

@register
def f1():
  print('running f1()')

@register
def f2():
  print('running f2()')

def f3():
  print('running f3()')


def main():
  print('running main()')
  print('registry', registry)
  f1()
  f2()
  f3()

if __name__ == '__main__':
  main()

running register(<function f1 at 0x7b815bc72b90>)
running register(<function f2 at 0x7b815bc71510>)
running main()
registry [<function f1 at 0x7b815bc72b90>, <function f2 at 0x7b815bc71510>]
running f1()
running f2()
running f3()


In [105]:
#07-3

promos = []

def promotion(promo_func):
  promos.append(promo_func)
  return promo_func

@promotion
def fidelity(order):
  return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * 0.1
  return discount

@promotion
def large_order_promo(order):
  distinct_items = {item.product for item in order.cart}
  if len(distinct_items) >= 10:
    return order.total() * 0.07
  return 0

def best_promo(order):
  return max(promo(order) for promo in promos)

In [106]:
#07-4
def f1(a):
  print(a)
  print(b)

f1(3)

3
9


In [107]:
b=6
f1(3)

3
6


In [108]:
#07-5
b=6
def f2(a):
  print(a)
  print(b)
  b =9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [109]:
#07-6

b=6
def f3(a):
  global b
  print(a)
  print(b)
  b=9

f3(3)

3
6


In [110]:
b

9

In [111]:
f3(3)

3
9


In [112]:
b=30
f3(3)

3
30


In [113]:
#07-7

class Averager():
  def __init__(self):
    self.series = []

  def __call__(self, new_value):
    self.series.append(new_value)
    total = sum(self.series)
    return total/len(self.series)

In [114]:
avg = Averager()
avg(10)

10.0

In [115]:
avg(11)

10.5

In [116]:
avg(12)

11.0

In [117]:
#07-8

def make_averager():
  series = []

  def averager(new_value):
    series.append(new_value)
    total = sum(series)
    return total/len(series)

  return averager

In [118]:
avg = make_averager()
avg(10)

10.0

In [119]:
avg(11)

10.5

In [120]:
avg(12)

11.0

In [121]:
#07-9
avg.__code__.co_varnames

('new_value', 'total')

In [122]:
avg.__code__.co_freevars

('series',)

In [123]:
avg.__closure__

(<cell at 0x7b815c4260b0: list object at 0x7b815bc59500>,)

In [124]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [125]:
def make_averager():
  count =0
  total =0

  def averager(new_value):
    count +=1
    total += new_value
    return total/count

  return averager

In [126]:
#07-10
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [127]:
#07-11
def make_averager():
  count =0
  total =0

  def averager(new_value):
    nonlocal count, total
    count +=1
    total += new_value
    return total/count

  return averager

In [128]:
avg = make_averager()
avg(10)

10.0

In [129]:
#07-12
import time

def clock(func):
  def clocked(*args):
    t0 = time.perf_counter()
    result = func(*args)
    elapsed = time.perf_counter()-t0
    name = func.__name__
    arg_str = ', '.join(repr(arg) for arg in args)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result
  return clocked

In [130]:
#07-13
@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
  print('*' * 40, 'Calling snooze(.123)')
  snooze(.123)
  print('*' * 40, 'Calling factorial(6)')
  print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12304570s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000065s] factorial(1) -> 1
[0.00042706s] factorial(2) -> 2
[0.00087971s] factorial(3) -> 6
[0.00128394s] factorial(4) -> 24
[0.00172417s] factorial(5) -> 120
[0.00214730s] factorial(6) -> 720
6! = 720


In [131]:
factorial.__name__

'clocked'

In [132]:
#07-14
import time
import functools

def clock(func):
  @functools.wraps(func)
  def clocked(*args, **kwargs):
    t0 = time.time()
    result = func(*args, **kwargs)
    elapsed = time.time() - t0
    arg_lst = []
    name = func.__name__
    if args:
      arg_lst.append(', '.join(repr(arg) for arg in args))
    if kwargs:
      pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
      arg_lst.append(', '.join(pairs))
    arg_str = ', '.join(arg_lst)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result
  return clocked

In [133]:
@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
  print('*' * 40, 'Calling snooze(.123)')
  snooze(.123)
  print('*' * 40, 'Calling factorial(6)')
  print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12323403s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000095s] factorial(1) -> 1
[0.00004220s] factorial(2) -> 2
[0.00007105s] factorial(3) -> 6
[0.00009608s] factorial(4) -> 24
[0.00012517s] factorial(5) -> 120
[0.00015616s] factorial(6) -> 720
6! = 720


In [134]:
factorial.__name__

'factorial'